## Imports

In [1]:
import copy
import logging
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

from model_merging.data.dataset import HFImageClassification
from model_merging.model.image_classifier import ImageClassifier
import open_clip
import wandb

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
from hydra import compose, initialize
from hydra.utils import instantiate
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig, OmegaConf
from torch.nn.utils import parameters_to_vector, vector_to_parameters

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import model_merging  # noqa
from model_merging.model.encoder import ClassificationHead, ImageEncoder
from model_merging.model.heads import (
    get_classification_head,
)
from model_merging.utils.io_utils import (
    boilerplate,
    load_model_from_hf,
)
from model_merging.utils.plots import plot_interactive_radar_chart
from model_merging.utils.utils import (
    build_callbacks,
    get_finetuning_accuracies,
    compute_avg_accuracy,
    print_memory,
)
import json
import os

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[19:40:06] INFO     PyTorch version 2.8.0 available.                                                   ]8;id=819522;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=177863;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py#54\54]8;;\

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/imports.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/data/datamodule.py:206: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "con

In [2]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="layer_analysis")
cfg = compose(config_name="multitask", overrides=["benchmark=N2"])

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/IPython/core/async_helpers.py:129: UserWarning: 
'hydra/launcher/basic' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
  coro.send(None)


## Boilerplate

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pylogger = logging.getLogger(__name__)

In [5]:
seed_index_everything(cfg)

logger, template_core = boilerplate(cfg)

num_tasks = len(cfg.benchmark.datasets)

# Temporarily disable struct mode to allow dynamic update
omegaconf.OmegaConf.set_struct(cfg, False)
cfg.num_tasks = num_tasks  # Now we can safely update it
omegaconf.OmegaConf.set_struct(cfg, True)  # Re-enable struct mode

# upperbound accuracies, used for logging the normalized accuracy
finetuned_accuracies: Dict[str, float] = get_finetuning_accuracies(
    cfg.misc.finetuned_accuracy_path
)[cfg.nn.encoder.model_name]

[19:40:15] INFO     Global seed set to 1608637542                                                        ]8;id=757;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py\seed.py]8;;\:]8;id=539100;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py#53\53]8;;\

           INFO     Setting seed 1608637542 from seeds[0]                                              ]8;id=506654;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=125355;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

           INFO     Tags: ['dev']                                                                       ]8;id=507214;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=552910;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#96\96]8;;\

           INFO     Instantiating <WandbLogger>                                                 ]8;id=346224;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=130475;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

           ERROR    Failed to detect the name of this notebook, you can set it manually with the     ]8;id=170108;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/wandb/jupyter.py\jupyter.py]8;;\:]8;id=507338;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/wandb/jupyter.py#224\224]8;;\
                    WANDB_NOTEBOOK_NAME environment variable to enable code saving.                                

wandb: Currently logged in as: crisostomi (gladia). Use `wandb login --relogin` to force relogin


[19:40:17] INFO     Uploading source code to W&B                                                ]8;id=635995;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=25814;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#63\63]8;;\

## Load models

In [6]:
# only has vision encoder, no text transformer
base_model: ImageEncoder = load_model_from_hf(
    model_name=cfg.nn.encoder.model_name
)

finetuned_models = {
    dataset.name: load_model_from_hf(
        model_name=cfg.nn.encoder.model_name, dataset_name=dataset.name
    )
    for dataset in cfg.benchmark.datasets
}

pylogger.info(f"Number of tasks: {cfg.num_tasks}")
pylogger.info(f"Finetuned models: {list(finetuned_models.keys())}")

finetuned_state_dicts = {
    dataset.name: finetuned_models[dataset.name].state_dict()
    for dataset in cfg.benchmark.datasets
}


[19:40:21] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=591644;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=663784;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=560796;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=355368;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[19:40:22] INFO     Removing text transformer from the model.                                         ]8;id=527597;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=234146;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

           INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=836233;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=748715;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=650903;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=922805;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

...

           INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=219438;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=568033;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=889220;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=474666;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[19:40:25] INFO     Removing text transformer from the model.                                         ]8;id=829130;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=168546;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

           INFO     Number of tasks: 2                                                             ]8;id=517586;file:///tmp/ipykernel_1901078/1349418912.py\1349418912.py]8;;\:]8;id=961760;file:///tmp/ipykernel_1901078/1349418912.py#13\13]8;;\

           INFO     Finetuned models: ['RESISC45', 'Cars']                                         ]8;id=756111;file:///tmp/ipykernel_1901078/1349418912.py\1349418912.py]8;;\:]8;id=152413;file:///tmp/ipykernel_1901078/1349418912.py#14\14]8;;\

## Training representation task vectors

### Get base representations

In [7]:
from tqdm import tqdm


base_representations: Dict[str, torch.Tensor] = {}

for dataset_cfg in cfg.benchmark.datasets:

    dataset = instantiate(
        dataset_cfg, preprocess_fn=base_model.val_preprocess
    )


    all_representations: List[torch.Tensor] = []
    base_model.eval().cuda()

    with torch.no_grad():
        for batch in tqdm(dataset.train_loader):
            images, labels = batch
            images = images.cuda()
            labels = labels.cuda()

            representations = base_model(images)
            all_representations.append(representations.cpu())

    all_representations_tensor = torch.cat(all_representations, dim=0)

    base_representations[dataset_cfg.name] = all_representations_tensor

100%|██████████| 64/64 [00:08<00:00,  7.17it/s]


### Get finetuned representations

In [8]:
from tqdm import tqdm

ft_representations: Dict[str, torch.Tensor] = {}

for dataset_cfg in cfg.benchmark.datasets:

    dataset = instantiate(
        dataset_cfg, preprocess_fn=base_model.val_preprocess
    )

    all_representations: List[torch.Tensor] = []

    ft_model = finetuned_models[dataset_cfg.name].cuda()

    with torch.no_grad():
        for batch in tqdm(dataset.train_loader):
            images, labels = batch
            images = images.cuda()
            labels = labels.cuda()

            representations = ft_model(images)
            all_representations.append(representations.cpu())

    all_representations_tensor = torch.cat(all_representations, dim=0)

    ft_representations[dataset_cfg.name] = all_representations_tensor

100%|██████████| 64/64 [00:09<00:00,  7.06it/s]


### Fit the transformation

In [9]:
task_transformations: Dict[str, torch.nn.Module] = {}
orthogonal_task_transformations: Dict[str, torch.nn.Module] = {}

for dataset_name, base_repr in base_representations.items():
    ft_repr = ft_representations[dataset_name]

    if base_repr.shape != ft_repr.shape:
        raise ValueError(
            f"Base and finetuned representations must have matching shapes, got {base_repr.shape} and {ft_repr.shape} for {dataset_name}."
        )

    base_double = base_repr.double()
    ft_double = ft_repr.double()

    solution = torch.linalg.lstsq(base_double, ft_double).solution.to(base_repr.dtype)

    linear = torch.nn.Linear(base_repr.shape[1], ft_repr.shape[1])
    with torch.no_grad():
        linear.weight.copy_(solution.T)
        linear.bias.zero_()

    task_transformations[dataset_name] = linear

    cross_covariance = base_double.T @ ft_double
    U, _, Vh = torch.linalg.svd(cross_covariance, full_matrices=False)
    orthogonal_matrix = (U @ Vh).to(base_repr.dtype)

    orthogonal_linear = torch.nn.Linear(
        base_repr.shape[1], ft_repr.shape[1], bias=False
    )
    with torch.no_grad():
        orthogonal_linear.weight.copy_(orthogonal_matrix.T)

    orthogonal_task_transformations[dataset_name] = orthogonal_linear


In [10]:
ft_representations['Cars'].shape

torch.Size([8144, 512])

In [11]:
class TransformedImageClassifier(ImageClassifier):
    def __init__(
        self,
        encoder: ImageEncoder,
        classifier: ClassificationHead,
        transform: torch.nn.Module,
        **kwargs,
    ):
        super().__init__(encoder, classifier, **kwargs)
        self.transform = transform

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        embeddings = self.encoder(x)

        embeddings = self.transform(embeddings)

        logits = self.classification_head(embeddings)

        return logits

### Evaluate

In [12]:
def evaluate_transformations(
    transformations: Dict[str, torch.nn.Module], label: str
) -> Dict[str, Any]:
    results = {}

    for dataset_cfg in cfg.benchmark.datasets:

        dataset = instantiate(
            dataset_cfg, preprocess_fn=base_model.val_preprocess
        )

        classification_head = get_classification_head(
            cfg.nn.encoder.model_name,
            dataset_cfg.name,
            ckpt_path=cfg.misc.ckpt_path,
            openclip_cachedir=cfg.misc.openclip_cachedir,
            device=cfg.device,
        )

        model = TransformedImageClassifier(
            encoder=base_model,
            classifier=classification_head,
            transform=transformations[dataset_cfg.name],
            x_key=cfg.conventions.x_key,
            y_key=cfg.conventions.y_key,
        )

        model.set_metrics(len(dataset.classnames))
        model.set_task(dataset_cfg.name)
        model.set_finetuning_accuracy(
            finetuned_accuracies[
                dataset_cfg.name + "Val" if cfg.eval_on_train else dataset_cfg.name
            ]
        )

        callbacks: List[Callback] = build_callbacks(
            cfg.train.callbacks, template_core
        )

        trainer = pl.Trainer(
            default_root_dir=cfg.core.storage_dir,
            plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
            logger=logger,
            callbacks=callbacks,
            limit_test_batches=(
                cfg.number_of_train_batches if cfg.eval_on_train else None
            ),
            **cfg.train.trainer,
        )
        pylogger.info(f"Evaluating {label} transform on the {dataset_cfg.name} test set!")
        test_results = trainer.test(model=model, dataloaders=dataset.test_loader)

        results[dataset_cfg.name] = test_results

    avg = compute_avg_accuracy(results)
    results["avg"] = [
        avg
    ]  # as a list for consistency due to lightning logging stuff this way

    prefixed_avg = {f"{label}_{metric}": value for metric, value in avg.items()}
    logger.experiment.log(prefixed_avg)

    pylogger.info({label: results})

    return results

linear_results = evaluate_transformations(task_transformations, "linear")
orthogonal_results = evaluate_transformations(
    orthogonal_task_transformations, "orthogonal"
)


Loading classification head from ./models//ViT-B-32/head_RESISC45.pt


/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recomm

[19:41:46] INFO     GPU available: True (cuda), used: True                                          ]8;id=717760;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=270984;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: TPU available: False, using: 0 TPU cores


           INFO     TPU available: False, using: 0 TPU cores                                        ]8;id=264469;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=104463;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

...

[19:42:25] INFO     You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores.  ]8;id=54522;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=589699;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                    To properly utilize them, you should set                                                       
                    `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off                   
                    precision for performance. For more details, read                                              
                    https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.ht                
                    ml#torch.set_float32_matmul_precision                                                          

           INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]                                            ]8;id=360678;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/accelerators/cuda.py\cuda.py]8;;\:]8;id=628872;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing DataLoader 0: 100%|██████████| 63/63 [00:08<00:00,  7.35it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       acc/test/Cars       │   0.006839945446699858    │
│      loss/test/Cars       │    15.990513801574707     │
│ normalized_acc/test/Cars  │   0.008569648489356041    │
└───────────────────────────┴───────────────────────────┘

[19:42:36] INFO     {'orthogonal': {'RESISC45': [{'acc/test/RESISC45': 0.018730157986283302,        ]8;id=939728;file:///tmp/ipykernel_1901078/439133286.py\439133286.py]8;;\:]8;id=308479;file:///tmp/ipykernel_1901078/439133286.py#63\63]8;;\
                    'loss/test/RESISC45': 7.381139755249023, 'normalized_acc/test/RESISC45':                       
                    0.01965356431901455}], 'Cars': [{'acc/test/Cars': 0.006839945446699858,                        
                    'loss/test/Cars': 15.990513801574707, 'normalized_acc/test/Cars':                              
                    0.008569648489356041}], 'avg': [{'acc/test/avg': 0.01278505171649158,                          
                    'normalized_acc/test/avg': 0.014111606404185295}]}}                                            